In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf

C:\Users\yangwa\AppData\Local\Continuum\miniconda2\envs\py36\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
from data import DataFeeder
files = ['data\\head_{}.gz'.format(i) for i in range(1, 6)]
data = DataFeeder(files, 10)
data_train = data.data_tensor(True)

feature size:6903


In [3]:
x = tf.placeholder(tf.float32, [None, 6903], 'features')
r = tf.placeholder(tf.float32, [None], 'rating')
# tf.nn.tanh
# name=scope
with tf.variable_scope('net'):
    # tf.initializers.glorot_normal()
    s = tf.layers.dense(x, units=1, activation=None, kernel_initializer=tf.truncated_normal_initializer(stddev=0.000000001), kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0004), name='layer_1')
with tf.variable_scope('loss'):
    # ground truth
    r_trans = tf.reshape(r, [1, -1])
    r_ = tf.reshape(r, [-1, 1])
    p = tf.sign(r_ - r_trans)
    # score
    s_trans = tf.reshape(s, [1, -1])
    diff = s - s_trans
    # loss
    #log_loss = tf.math.softplus(-(p * diff))
    log_loss = tf.nn.relu(-(p * diff))
    log_loss = tf.reduce_sum(log_loss)


In [4]:
sess = tf.Session()
features = sess.run(data_train)

epoch 0
(epoch 0) reading data\head_2.gz


In [13]:
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [5]:
feed_dict = {
    x: features['features'],
    r: features['rating']
}

In [6]:
print (features['features'].shape)

(10, 6903)


In [17]:
result = sess.run(p, feed_dict=feed_dict)
print (result)

[[ 0. -1. -1. -1. -1. -1. -1.  0. -1. -1.]
 [ 1.  0. -1.  1.  1. -1. -1.  1. -1.  1.]
 [ 1.  1.  0.  1.  1.  0.  0.  1.  0.  1.]
 [ 1. -1. -1.  0. -1. -1. -1.  1. -1.  0.]
 [ 1. -1. -1.  1.  0. -1. -1.  1. -1.  1.]
 [ 1.  1.  0.  1.  1.  0.  0.  1.  0.  1.]
 [ 1.  1.  0.  1.  1.  0.  0.  1.  0.  1.]
 [ 0. -1. -1. -1. -1. -1. -1.  0. -1. -1.]
 [ 1.  1.  0.  1.  1.  0.  0.  1.  0.  1.]
 [ 1. -1. -1.  0. -1. -1. -1.  1. -1.  0.]]


In [18]:
features['rating']

array([  0.,  10.,  30.,   8.,   9.,  30.,  30.,   0.,  30.,   8.], dtype=float32)

In [19]:
result = sess.run(s, feed_dict=feed_dict)
print (result)

[[ 0.05618896]
 [-6.91015816]
 [ 0.02507003]
 [ 0.04548552]
 [-0.47460976]
 [ 8.67575455]
 [-1.09109664]
 [ 0.05251767]
 [ 0.03010756]
 [ 0.05273222]]


In [20]:
result = sess.run(diff, feed_dict=feed_dict)
print (result)

[[  0.00000000e+00   6.96634722e+00   3.11189331e-02   1.07034408e-02
    5.30798733e-01  -8.61956596e+00   1.14728558e+00   3.67128477e-03
    2.60814019e-02   3.45673785e-03]
 [ -6.96634722e+00   0.00000000e+00  -6.93522835e+00  -6.95564365e+00
   -6.43554831e+00  -1.55859127e+01  -5.81906128e+00  -6.96267605e+00
   -6.94026566e+00  -6.96289015e+00]
 [ -3.11189331e-02   6.93522835e+00   0.00000000e+00  -2.04154924e-02
    4.99679804e-01  -8.65068436e+00   1.11616671e+00  -2.74476483e-02
   -5.03753126e-03  -2.76621953e-02]
 [ -1.07034408e-02   6.95564365e+00   2.04154924e-02   0.00000000e+00
    5.20095289e-01  -8.63026905e+00   1.13658214e+00  -7.03215599e-03
    1.53779611e-02  -7.24670291e-03]
 [ -5.30798733e-01   6.43554831e+00  -4.99679804e-01  -5.20095289e-01
    0.00000000e+00  -9.15036392e+00   6.16486907e-01  -5.27127445e-01
   -5.04717350e-01  -5.27341962e-01]
 [  8.61956596e+00   1.55859127e+01   8.65068436e+00   8.63026905e+00
    9.15036392e+00   0.00000000e+00   9.76685